<a href="https://colab.research.google.com/github/soares-f/JMeSH/blob/main/Inverted_index_from_LitCovid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Extracting LitCovid in local env




In [1]:
!wget https://ftp.ncbi.nlm.nih.gov/pub/lu/LitCovid/litcovid2pubtator.xml.gz

--2021-01-19 13:48:28--  https://ftp.ncbi.nlm.nih.gov/pub/lu/LitCovid/litcovid2pubtator.xml.gz
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.13, 2607:f220:41e:250::10, 2607:f220:41e:250::11, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.13|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 528266354 (504M) [application/x-gzip]
Saving to: ‘litcovid2pubtator.xml.gz’

litcovid2pubtator.x 100%[===================>] 503.79M  70.6MB/s    in 6.3s    

2021-01-19 13:48:34 (79.8 MB/s) - ‘litcovid2pubtator.xml.gz’ saved [528266354/528266354]



In [2]:
!gunzip litcovid2pubtator.xml.gz

In [3]:
!head litcovid2pubtator.xml

<?xml version='1.0' encoding='UTF-8'?><!DOCTYPE collection SYSTEM 'BioC.dtd'>
<!-- 
*
*                            PUBLIC DOMAIN NOTICE
*               National Center for Biotechnology Information
*
*  This software/database is a "United States Government Work" under the
*  terms of the United States Copyright Act.  It was written as part of
*  the author's official duties as a United States Government employee and


In [2]:
pip install bioc

In [1]:
filename = 'litcovid2pubtator.xml'

In [139]:
from nltk import tokenize
nltk.download('punkt')
import collections
import uuid 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
import bioc
# read from a ByteIO
with open(filename, 'rb') as fp:
    reader = bioc.BioCXMLDocumentReader(fp)
    collection_info = reader.get_collection_info()
    for document in reader:
        print(document)
        

In [42]:
document.passages[1].annotations[2]

BioCAnnotation[id=18,text='pneumonia',infons=[Identifier=MESH:D011014,type=Disease],locations=[BioCLocation[offset=138,length=9]],]

In [142]:
def get_lens_passage(passage):
  sentences = tokenize.sent_tokenize(passage.text)
  lens_sentences = []
  ids_sentences = []
  for sentence in sentences:
    lens_sentences.append(len(sentence))
    ids_sentences.append(uuid.uuid4())
  return(lens_sentences,sentences, ids_sentences)

In [68]:
def extract_annotations_passage(passage, type_of_annotation='MESH'):
  annotations_processed = []
  for annotation in passage.annotations:
    tmp =  {k.lower(): v for k, v in annotation.infons.items()}
    if type_of_annotation in tmp['identifier']:
      identifier = annotation.infons['identifier']
      type_of_identifier = annotation.infons['type']
      locations = []
      for location in annotation.locations:
        locations.append(location.offset)
      text = annotation.text
      annotations_processed.append({'identifier': identifier,
                                    'type_of_identifier': type_of_identifier,
                                    'locations': locations,
                                    'text': text})
  if len(annotations_processed) > 0:
    return(annotations_processed)
  else:
    return None

In [156]:
def match_annotations_sentences(passage, file_handler=None):
  annotations_processed = extract_annotations_passage(passage)
  lens_sentences, sentences,ids_sentences = get_lens_passage(passage)
  if file_handler is not None:
    for i in range(len(sentences)):
      file_handler.write(str(ids_sentences[i]) + '\t' + sentences[i])
  else:
    for i in range(len(sentences)):
      print(str(ids_sentences[i]) + '\t' + sentences[i])
  dict_annotations = collections.defaultdict(set)
  offsets_sentences = []
  for i in range(len(lens_sentences)):
    if i == 0:
      offsets_sentences.append(lens_sentences[i]) 
    else:
      offsets_sentences.append(lens_sentences[i]+offsets_sentences[i-1])
  for annotation in annotations_processed:
    min_location = min(annotation['locations'])
    #print(min_location)
    location_annotation = next((x[0] for x in enumerate(offsets_sentences) if x[1] >= min_location), len(offsets_sentences)-1)
    dict_annotations[annotation['identifier']].add(str(ids_sentences[location_annotation]))
  print(dict_annotations)


In [117]:
tmp_passage1 = extract_annotations_passage(document.passages[1])

In [131]:
lens,_ = get_lens_passage(document.passages[1])

In [157]:
match_annotations_sentences(document.passages[1])

0c0e34a1-b961-4866-96f3-8b4794a76fa0	Novel coronavirus disease 2019 (COVID-19) is known to cause severe pneumonia and acute respiratory distress syndrome which may lead to death.
e14da967-1b57-4f3b-816d-0068f1a01914	Several treatments have been tested in the race to find a treatment regimen for this deadly viral infection.
7136ab30-5879-464f-8c67-ee39f5b4647c	Tocilizumab, a recombinant humanized anti-interleukin-6 receptor monoclonal antibody, has been used and found to be beneficial in patients with COVID-19 and in cytokine storm.
afd8be1b-68b3-4773-bdb3-6a80a3c574e9	We present the case of a young, otherwise healthy male, presenting with COVID-19 and successfully treated in the intensive care unit with tocilizumab.
defaultdict(<class 'set'>, {'MESH:C000657245': {'afd8be1b-68b3-4773-bdb3-6a80a3c574e9', '0c0e34a1-b961-4866-96f3-8b4794a76fa0'}, 'MESH:D011014': {'0c0e34a1-b961-4866-96f3-8b4794a76fa0'}, 'MESH:D012128': {'e14da967-1b57-4f3b-816d-0068f1a01914'}, 'MESH:D003643': {'e14da967-1b